# Micro-benchmarks and timeit

In [1]:
import io

def build_concat(strings):
    result = ''
    for s in strings:
        result += s
    return result

def build_join(strings):
    return ''.join(strings)

def build_sio(strings):
    sio = io.StringIO()
    for s in strings:
        sio.write(s)
    return sio.getvalue()

strings = [str(i) for i in range(1000)]

In [2]:
build_concat(strings) == build_join(strings) == build_sio(strings) 

True

In [3]:
import timeit

In [9]:
for fname in ('concat', 'join', 'sio'):
    elapsed = timeit.timeit(
        'build_%s(strings)' % fname, 
        globals=globals(),
        number=10_000)
    print(fname, elapsed)

concat 0.5870490000233985
join 0.06981250003445894
sio 0.6017359999823384


## Jupyter Notebook (and IPython) provide a nice helper

We can use `%timeit` (or usually just `timeit`) to run a version of `timeit` in IPython

In [5]:
%%timeit 
build_concat(strings)

56.4 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%timeit build_join(strings)

7.06 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
%timeit build_sio(strings)

63.8 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Results of our microbenchmark

- `''.join()` is the fastest, around 10x faster than concatenating strings
- You should use `''.join` when you need to build a big string from a list of strings

# Profiling Python Code

You can profile an entire script by running it via `python -m cProfile` to get summary information about the whole thing:

In [10]:
pwd

'/home/rick446/src/arborian-classes/src'

In [11]:
%%file data/profiling/profiletest.py
import re

text = '''The quick brown fox jumps over the lazy dog'''
for x in range(10_000):
    re.search('fox', text)

Overwriting data/profiling/profiletest.py


In [12]:
!python -m cProfile data/profiling/profiletest.py

         40096 function calls (40095 primitive calls) in 0.009 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 enum.py:283(__call__)
        2    0.000    0.000    0.000    0.000 enum.py:562(__new__)
        1    0.000    0.000    0.000    0.000 enum.py:833(__and__)
        1    0.002    0.002    0.009    0.009 profiletest.py:1(<module>)
    10000    0.002    0.000    0.007    0.000 re.py:198(search)
    10000    0.002    0.000    0.003    0.000 re.py:289(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:432(_generate_overlap_table)
        1    0.000    0.000    0.000    0.000 sre_compile.py:453(_get_iscased)
        1    0.000    0.000    0.000    0.000 sre_compile.py:461(_get_literal_prefix)
        1    0.000    0.000    0.000    0.000 sre_compile.py:536(_compile_info)
        2    0.000    0.000    0.000    0.000 sre_compile.py:595(isstring)
        1  

In [13]:
!python -m cProfile --help

Usage: cProfile.py [-o output_file_path] [-s sort] [-m module | scriptfile] [arg] ...

Options:
  -h, --help            show this help message and exit
  -o OUTFILE, --outfile=OUTFILE
                        Save stats to <outfile>
  -s SORT, --sort=SORT  Sort order when printing to stdout, based on
                        pstats.Stats class
  -m                    Profile a library module


In [14]:
!python -m cProfile -s time data/profiling/profiletest.py

         40096 function calls (40095 primitive calls) in 0.010 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.003    0.000    0.008    0.000 re.py:198(search)
    10000    0.003    0.000    0.004    0.000 re.py:289(_compile)
        1    0.002    0.002    0.010    0.010 profiletest.py:1(<module>)
    10000    0.001    0.000    0.001    0.000 {method 'search' of 're.Pattern' objects}
    10009    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 sre_compile.py:759(compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:536(_compile_info)
        1    0.000    0.000    0.000    0.000 sre_parse.py:937(parse)
        3    0.000    0.000    0.000    0.000 sre_parse.py:172(append)
        1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        1   

In [15]:
%%file data/profiling/profiletest.py
import re

text = '''The quick brown fox jumps over the lazy dog'''
my_regex = re.compile('fox')
for x in range(10_000):
    my_regex.search(text)

Overwriting data/profiling/profiletest.py


In [16]:
!python -m cProfile -s time data/profiling/profiletest.py 

         10099 function calls (10098 primitive calls) in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.002    0.002 profiletest.py:1(<module>)
    10000    0.001    0.000    0.001    0.000 {method 'search' of 're.Pattern' objects}
        1    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 re.py:289(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:759(compile)
        3    0.000    0.000    0.000    0.000 sre_parse.py:172(append)
        1    0.000    0.000    0.000    0.000 sre_compile.py:536(_compile_info)
        1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        1    0.000    0.000    0.000    0.000 sre_parse.py:937(parse)
        4    0.000    0.000    0.000    0.000 sre_parse.py:233(__next)
        1    0.000    0.000    0.000    0.000 sre_parse.py:435(_parse_sub)
       21    0

In [17]:
%run -p data/profiling/profiletest.py

In [18]:
!python -m cProfile -s time -o profile-stats data/profiling/profiletest.py

### Direct profiling

We can also profile just a few Python statements or a function:

In [19]:
import re
import cProfile

In [20]:
cProfile.run('re.compile("foo|bar")', sort='time')

         214 function calls (207 primitive calls) in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 re.py:289(_compile)
      3/1    0.000    0.000    0.000    0.000 sre_compile.py:71(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:536(_compile_info)
        1    0.000    0.000    0.000    0.000 sre_compile.py:759(compile)
        1    0.000    0.000    0.000    0.000 sre_parse.py:937(parse)
        1    0.000    0.000    0.000    0.000 sre_parse.py:435(_parse_sub)
        1    0.000    0.000    0.000    0.000 sre_compile.py:276(_optimize_charset)
      3/1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
       18    0.000    0.000    0.000    0.000 sre_parse.py:164(__getitem__)
       48

We can also save the statistics to a file for analysis later:

In [21]:
cProfile.run('re.compile("foo|bar")', 're-stats')

For analysis, we use the `pstats` module:

In [22]:
import pstats

In [23]:
p = pstats.Stats('profile-stats')

In [24]:
p.print_stats()

Tue Jul 13 17:21:15 2021    profile-stats

         10099 function calls (10098 primitive calls) in 0.002 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       21    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
       10    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
    13/12    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.ord}
        1    0.000    0.000    0.000    0.000 {built-in method _sre.compile}
    10000    0.001    0.000    0.001    0.000 {method 'search' o

In [25]:
p.strip_dirs()
p.print_stats()

Tue Jul 13 17:21:15 2021    profile-stats

         10099 function calls (10098 primitive calls) in 0.002 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       21    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
       10    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
    13/12    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.ord}
        1    0.000    0.000    0.000    0.000 {built-in method _sre.compile}
    10000    0.001    0.000    0.001    0.000 {method 'search' o

In [26]:
p.sort_stats('time')

In [27]:
p.print_stats(10)

Tue Jul 13 17:21:15 2021    profile-stats

         10099 function calls (10098 primitive calls) in 0.002 seconds

   Ordered by: internal time
   List reduced from 41 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.002    0.002 profiletest.py:1(<module>)
    10000    0.001    0.000    0.001    0.000 {method 'search' of 're.Pattern' objects}
        1    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 sre_compile.py:536(_compile_info)
        1    0.000    0.000    0.000    0.000 re.py:289(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:759(compile)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        3    0.000    0.000    0.000    0.000 sre_parse.py:172(append)
        1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        1    0.000    0.000    0.000    0.000 sre_parse

Jupyter / IPython also has a magic function to help us here, as well:

In [28]:
%prun for x in range(10000): re.compile('foo|bar')

In [29]:
%%prun 
for x in range(10000): 
    re.compile('foo|bar')

# Instrumentation 

In [30]:
p = cProfile.Profile()

In [31]:
p.enable()
for x in range(10000):
    re.compile('re|foo')
    lst0 = range(100)
    lst1 = list(range(100))
p.disable()

In [32]:
p.print_stats(sort='time')

         30242 function calls (30235 primitive calls) in 0.021 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.020    0.020 <ipython-input-31-97b44869a5b7>:2(<module>)
    10000    0.003    0.000    0.004    0.000 re.py:289(_compile)
    10000    0.002    0.000    0.006    0.000 re.py:250(compile)
    10022    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.021    0.010 interactiveshell.py:3376(run_code)
        1    0.000    0.000    0.000    0.000 <ipython-input-31-97b44869a5b7>:6(<module>)
      3/1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        2    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 sre_parse.py:921(fix_flags)
      3/1    0.000    0.000    0.000    0.000 sre_c

In [33]:
with p:
    re.compile('re|foo')
    lst0 = range(100)
    lst1 = list(range(100))
p.print_stats(sort='time')

         30247 function calls (30240 primitive calls) in 0.021 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.020    0.020 <ipython-input-31-97b44869a5b7>:2(<module>)
    10001    0.003    0.000    0.004    0.000 re.py:289(_compile)
    10001    0.002    0.000    0.006    0.000 re.py:250(compile)
    10023    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.021    0.010 interactiveshell.py:3376(run_code)
        1    0.000    0.000    0.000    0.000 <ipython-input-31-97b44869a5b7>:6(<module>)
      3/1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        2    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 sre_parse.py:921(fix_flags)
      3/1    0.000    0.000    0.000    0.000 sre_c

(Profilers are already context managers in Python 3.8)

In [34]:
from contextlib import contextmanager

@contextmanager
def profiler(p):
    p.enable()
    try:
        yield p
    finally:
        p.disable()

In [35]:
with profiler(p):
    for x in range(10000):
        re.compile('re|foo')
        lst0 = range(100)
        lst1 = list(range(100))

In [36]:
p.print_stats(sort='time')

         60251 function calls (60244 primitive calls) in 0.027 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.020    0.020 <ipython-input-31-97b44869a5b7>:2(<module>)
    20001    0.006    0.000    0.009    0.000 re.py:289(_compile)
    20001    0.004    0.000    0.013    0.000 re.py:250(compile)
    20023    0.003    0.000    0.003    0.000 {built-in method builtins.isinstance}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.021    0.010 interactiveshell.py:3376(run_code)
        1    0.000    0.000    0.000    0.000 <ipython-input-31-97b44869a5b7>:6(<module>)
      3/1    0.000    0.000    0.000    0.000 sre_parse.py:174(getwidth)
        2    0.000    0.000    0.000    0.000 sre_parse.py:493(_parse)
        1    0.000    0.000    0.000    0.000 sre_parse.py:921(fix_flags)
        3    0.000    0.000    0.000    0.000 conte

# Instrumenting high-performance code

There are times when we want to profile, but we don't want to incur the performance penalty. For instance, we might want to see the profile of a running production system, without impacting its performance in a major way.

For that, we can profile a _sample_ of the calls to a function.

In [37]:
import random
import functools

def instrument(profiler, probability=0.10):
    '''Profile some of the calls to the decorated function.
    
    The default probability of profiling a call is 10%.
    '''
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if random.random() < probability:
#                 with profiler:  #if py38+
#                     return func(*args, **kwargs)
                try:
                    profiler.enable()
                    return func(*args, **kwargs)
                finally:
                    profiler.disable()
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator

In [38]:
import cProfile
prof = cProfile.Profile()

In [39]:
@instrument(prof, 0.2)
def build_join(strings):
    return ''.join(strings)

In [40]:
for x in range(10_000): 
    build_join(strings)

In [41]:
prof.print_stats(sort='time')

         5847 function calls in 0.015 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1949    0.014    0.000    0.014    0.000 {method 'join' of 'str' objects}
     1949    0.001    0.000    0.015    0.000 <ipython-input-39-4e16200570cb>:1(build_join)
     1949    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [42]:
from contextlib import contextmanager

@contextmanager
def profiling(profiler, probability=0.10):
    if random.random() < probability:
#         with profiler:
#             yield profiler
        try:
            profiler.enable()
            yield profiler
        finally:
            profiler.disable()
    else:
        yield None

In [43]:
%timeit random.random() < 0.10

74.5 ns ± 1.54 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [45]:
prof = cProfile.Profile()
prof1 = cProfile.Profile()

num_profiles = 0
for x in range(10_000): 
    with profiling(prof, 0.02) as as_value:
        with profiling(prof1, 0.5) as as_value1:
            # If profiling, as_value == prof
            # If NOT profiling, as_value == None
            if as_value:
                num_profiles += 1
            ''.join(strings)

prof.print_stats(sort='time')

         2343 function calls in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       96    0.001    0.000    0.001    0.000 {method 'enable' of '_lsprof.Profiler' objects}
      105    0.001    0.000    0.001    0.000 {method 'join' of 'str' objects}
      411    0.000    0.000    0.001    0.000 <ipython-input-42-549dd9f2ece6>:3(profiling)
      201    0.000    0.000    0.000    0.000 contextlib.py:82(__init__)
      210    0.000    0.000    0.000    0.000 contextlib.py:117(__exit__)
      201    0.000    0.000    0.000    0.000 contextlib.py:238(helper)
      411    0.000    0.000    0.001    0.000 {built-in method builtins.next}
      201    0.000    0.000    0.001    0.000 contextlib.py:108(__enter__)
      201    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
      201    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}
      105    0.000    0.000    0.000    0.

In [49]:
prof1.print_stats(sort='time')

         25435 function calls in 0.038 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5087    0.035    0.000    0.035    0.000 {method 'join' of 'str' objects}
     5087    0.001    0.000    0.003    0.000 contextlib.py:117(__exit__)
     5087    0.001    0.000    0.002    0.000 <ipython-input-42-549dd9f2ece6>:3(profiling)
     5087    0.001    0.000    0.002    0.000 {built-in method builtins.next}
     5087    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [47]:
num_profiles

201

# Lab

Open the [profiling lab][profiling_lab]

[profiling_lab]: ./profiling-lab.ipynb